In [ ]:
import pandas as pd

Données sur le PIB


Nom du dataframe: df_GDP

Les pays retenus sont ceux pour lesquels on a réussi à collecter toutes les données relatives aux variables d'intérêt: 


Autriche (Austria)
Belgique (Belgium)
Bulgarie (Bulgaria)
Tchéquie (Czechia)
Danemark (Denmark)
Estonie (Estonia)
Finlande (Finland)
France (France)
Allemagne (Germany)
Grèce (Greece)
Hongrie (Hungary)
Islande (Iceland)
Irlande (Ireland)
Italie (Italy)
Lettonie (Latvia)
Lituanie (Lithuania)
Luxembourg (Luxembourg)
Pays-Bas (Netherlands)
Norvège (Norway)
Pologne (Poland)
Portugal (Portugal)
Roumanie (Romania)
Slovaquie (Slovakia)
Slovénie (Slovenia)
Espagne (Spain)
Suède (Sweden)
Suisse (Switzerland)
Royaume-Uni (United Kingdom)


In [ ]:
#Création d'une liste comportant les pays choisis
countries_selected=[
    "Austria", "Belgium", "Bulgaria", "Czechia", "Denmark", "Estonia", 
    "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", 
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Netherlands", 
    "Norway", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", 
    "Spain", "Sweden", "Switzerland", "United Kingdom"
]


Cette copie comporte des data frames similaires à ceux du premier document. Seule différence: les indices comportent les dates (apparemment c'est mieux pour une analyse en séries temporelles)

In [ ]:
import pandas as pd

# Extraction des données relatives au GDP (Données Eurostat)
'''
Informations générales
Online data code:namq_10_gdp
Unit of measure: Current prices-million euro
'''

# Charger les données
df_GDP = pd.read_csv("Données_extraites/GDP_trimestriel_eurostat.csv", encoding='utf-8')
#print(df_GDP.head())

# Choix des colonnes rélévantes
df_GDP_selected = df_GDP[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_GDP_pivot = df_GDP_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure un préfixe "PIB_" suivi du nom du pays
df_GDP_pivot.columns = [f'PIB_{col}' for col in df_GDP_pivot.columns]

# On filtre les pays qui nous intéressent
colonnes_a_garder = [f'PIB_{pays}' for pays in countries_selected]

# Vérification et filtrage
df_GDP=df_GDP_pivot
df_GDP = df_GDP[[col for col in df_GDP.columns if col in colonnes_a_garder]]
df_GDP




Dans ce qui suit: nous allons essayer de concatener trois datasets:
- 2 comportant des données trimestrielles (1998-2008 et 2008-2024)
- 1 comporatant des données annuelles (1995-1998)

Nous avons gardé la même source (Eurostat) afin d'avoir des estimations homogènes


In [ ]:
import pandas as pd

# Extraction des données Working Hours per year (Données Eurostat: 1995-1997)

'''
Informations générales
Lien: https://doi.org/10.2908/LFSA_EWHANA
Code: lfsa_ewhana
'''

# Charger les données
df_working_hours_eurostat_an = pd.read_csv("Données_extraites/Working_hours_eurostat_1995-1997.csv", encoding='utf-8')
#print(df_working_hours_eurostat_an.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_an = df_working_hours_eurostat_an[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_working_hours_selected_eurostat_an.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_an = df_working_hours_selected_eurostat_an.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE')

# Renommer les colonnes pour inclure un préfixe "WorkingHours_" suivi du nom du pays
df_working_hours_pivot_eurostat_an.columns = [f'Working_Hours_{col}' for col in df_working_hours_pivot_eurostat_an.columns]

# Vérifier que les dates sont bien en index et les pays sont renommés
#print(df_working_hours_pivot_eurostat_an.head())
df_working_hours_pivot_eurostat_an


In [ ]:
# transformation: données annuelles --> trimestrielles // Option 1: recopier les données annuelles 4 fois

# Création d'une copie du dataframe original
df_working_hours_quarterly_1 = df_working_hours_pivot_eurostat_an.copy()

# Répéter chaque année 4 fois
df_working_hours_quarterly_1 = df_working_hours_pivot_eurostat_an.loc[df_working_hours_pivot_eurostat_an.index.repeat(4)]

# Générer un nouvel index trimestriel sous le format 'YYYY-QX'
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
df_working_hours_quarterly_1['TIME_PERIOD'] = [f"{year}-{quarter}" for year in df_working_hours_pivot_eurostat_an.index for quarter in quarters]

# Remettre 'TIME_PERIOD' comme index
df_working_hours_quarterly_1 = df_working_hours_quarterly_1.set_index('TIME_PERIOD')

# Renommer les colonnes pour éviter la duplication des préfixes dans les noms des colonnes
df_working_hours_quarterly_1.columns = [col.replace("Working_Hours_Working_Hours", "Working_Hours") for col in df_working_hours_quarterly_1.columns]

# Vérification
df_working_hours_quarterly_1






In [ ]:
#transformation: données annuelles--> trimestrielles // Option 2: Interpolation
# Création d'une copie du dataframe original
df_working_hours_quarterly_2 = df_working_hours_pivot_eurostat_an.copy()


# Idée pour l'interpolation: recopier la valeur annuelle pour le premier trimestre et
# ajouter des lignes "vides" entre le premier trimestre des deux années consécutives, puis faire l'interpolation
new_index = []
values = []

for year in df_working_hours_quarterly_2.index:
    # Ajouter Q1 pour chaque année (avec la valeur annuelle)
    new_index.append(f"{year}-Q1")
    values.append(df_working_hours_quarterly_2.loc[year].values)
    
    # Ajouter Q2, Q3 et Q4 (lignes vides pour l'instant)
    for quarter in ['Q2', 'Q3', 'Q4']:
        new_index.append(f"{year}-{quarter}")
        values.append([None] * len(df_working_hours_quarterly_2.columns))

# Création d'un nouveau DataFrame trimestriel avec les mêmes colonnes
df_working_hours_quarterly_2 = pd.DataFrame(values, index=new_index, columns=df_working_hours_quarterly_2.columns)

#On met le nom de "TIME_PERIOD"
df_working_hours_quarterly_2.index.name = 'TIME_PERIOD'

# Interpolation linéaire
df_working_hours_quarterly_2 = df_working_hours_quarterly_2.interpolate(method='linear')

# Renommer les colonnes pour éviter la duplication des préfixes dans les noms des colonnes
df_working_hours_quarterly_2.columns = [col.replace("Working_Hours_Working_Hours", "Working_Hours") for col in df_working_hours_quarterly_1.columns]

#Vérification
df_working_hours_quarterly_2


In [ ]:
# Extraction des données Working Hours trimestriels (Données Eurostat à partir de 2008)
'''
Informations générales: données trimestrielles à partir de 2008
Online data code: lfsq_ewhan2
Unit of measure: Hours per year per person
'''

# Charger les données
df_working_hours_eurostat_1 = pd.read_csv("Données_extraites/Working_hours_eurostat_2008-2024.csv", encoding='utf-8')
#print(df_working_hours_eurostat_1.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_1 = df_working_hours_eurostat_1[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_working_hours_selected_eurostat_1.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_1 = df_working_hours_selected_eurostat_1.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure un préfixe "WorkingHours_" suivi du nom du pays
df_working_hours_pivot_eurostat_1.columns = [f'Working_Hours_{col}' for col in df_working_hours_pivot_eurostat_1.columns]

# Vérifier que les dates sont bien en index et les pays sont renommés
#print(df_working_hours_pivot_eurostat_1.head())
df_working_hours_pivot_eurostat_1

#On change le nom pour simplifier
df_working_hours_eurostat_1=df_working_hours_pivot_eurostat_1
df_working_hours_eurostat_1


In [ ]:
# Extraction des données Working Hours per year (Données Eurostat à partir de 1998-2008)
'''
Informations générales: données trimestrielles 1998--> 2008
Online data code: lfsq_ewhana
Source: https://ec.europa.eu/eurostat/databrowser/view/lfsq_ewhana__custom_15382513/default/table?lang=en
Unit of measure: Hours per year per person
'''

# Charger les données
df_working_hours_eurostat_2 = pd.read_csv("Données_extraites/Working_hours_eurostat_1998-2007.csv", encoding='utf-8')
#print(df_GDP.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_2 = df_working_hours_eurostat_2[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_2 = df_working_hours_selected_eurostat_2.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure un préfixe (par exemple, "Working_Hours_")
df_working_hours_pivot_eurostat_2.columns = [f'Working_Hours_{col}' for col in df_working_hours_pivot_eurostat_2.columns]

# Supprimer les 4 dernières lignes (pour que l'année 2008 ne se répète pas 2 fois lors de la concaténation finale)
df_working_hours_pivot_eurostat_2 = df_working_hours_pivot_eurostat_2.iloc[:-4, :]

#On change le nom pour simplifier
df_working_hours_eurostat_2=df_working_hours_pivot_eurostat_2
df_working_hours_eurostat_2


Ainsi nous obtenons les données sur les Working Hours

Nom du data_frame: df_working_hours

In [ ]:
# Concaténation verticale pour fusionner les trois jeux de données 
# Remarque: pour la période 1995-1997, nous avons pris la méthode par l'interpolation linéaire

df_working_hours = pd.concat(
    [df_working_hours_quarterly_2, df_working_hours_pivot_eurostat_2, df_working_hours_pivot_eurostat_1], 
    axis=0
)

df_working_hours

# On filtre les pays qui nous intéressent
colonnes_a_garder = [f'Working_Hours_{pays}' for pays in countries_selected]

# Vérification et filtrage
df_working_hours = df_working_hours[[col for col in df_working_hours.columns if col in colonnes_a_garder]]
df_working_hours



Données des taux d'intérêts

Taux court terme : "Les données se réfèrent aux bons acceptés par la banque/Certificats de dépôt négociables-3 mois."
Nom du dataframe: df_ST_IR


Taux long terme : "Les données se réfèrent au taux de rendement estimé en clôture des obligations du trésor à 10 ans."
Nom du dataframe: df_LT_IR

In [ ]:
# Pour le long terme (une possibilité): Long term interest rates
'''
Source: https://data-explorer.oecd.org/vis?lc=en&pg=0&tm=LONG-TERM%20INTEREST
Unit of measure: Percent per annum
'''

# Charger les données
df_LT_IR = pd.read_csv('Données_extraites/Long_term_IR_OCDE.csv', encoding='utf-8')

# Choix des colonnes rélévantes
df_LT_IR_selected = df_LT_IR[df_LT_IR['Reference area'].isin(countries_selected)][['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]

# Obtention des données de panel en format séries temporelles
df_LT_IR_pivot = df_LT_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure le préfixe LT_IR_ (=Long Term Interest Rate)
df_LT_IR_pivot.columns = [f'LT_IR_{col}' for col in df_LT_IR_pivot.columns]
df_LT_IR_pivot = df_LT_IR_pivot.rename(columns={'LT_IR_Slovak Republic': 'LT_IR_Slovakia'})

# On filtre les pays qui nous intéressent
colonnes_a_garder = [f'LT_IR_{pays}' for pays in countries_selected]

# Vérification et filtrage
df_LT_IR=df_LT_IR_pivot
df_LT_IR


In [ ]:
# Pour le court terme (une possibilité): short term interest rates

'''
Informations générales 
source: https://data-explorer.oecd.org/vis?lc=en&pg=0&tm=LONG-TERM%20INTEREST&snb=21&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_STES%40DF_FINMARK&df[ag]=OECD.SDD.STES&df[vs]=4.0&dq=GBR%2BROU%2BHRV%2BBGR%2BESP%2BTUR%2BCHE%2BSWE%2BSVN%2BSVK%2BPRT%2BPOL%2BNOR%2BNLD%2BLUX%2BLTU%2BLVA%2BITA%2BIRL%2BISL%2BHUN%2BGRC%2BDEU%2BFRA%2BFIN%2BEST%2BDNK%2BCZE%2BBEL%2BAUT%2BAUS.Q.IRLT.PA.....&to[TIME_PERIOD]=false&pd=1995-Q1%2C
Unit of measure: Percent per annum
'''

df_ST_IR=pd.read_csv('Données_extraites/Short_term_IR_OCDE.csv', encoding='utf-8')

# Choix des colonnes rélévantes
df_ST_IR_selected = df_ST_IR[['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]

# Obtention des données de panel en format séries temporelles
df_ST_IR_pivot = df_ST_IR_selected.pivot_table(index='TIME_PERIOD', columns='Reference area', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure le préfixe ST_IR_ (=Short Term Interest Rate)
df_ST_IR_pivot.columns = [f'ST_IR_{col}' for col in df_ST_IR_pivot.columns]
df_ST_IR_pivot = df_ST_IR_pivot.rename(columns={'ST_IR_Slovak Republic': 'ST_IR_Slovakia'})


# On simplifie le nom, et on vérifie// pas besoin de filtrer car les 28 pays sont ceux qui nous intéressent
df_ST_IR=df_ST_IR_pivot
df_ST_IR



Import du CPI

Nom du dataframe: 
df_CPI


In [ ]:
# données téléchargées depuis le site de eurostat
# Décompresser le fichier qui était en .gz

df_CPI=pd.read_csv('Données_extraites/CPI_annual_eurostat_1996-2024.csv', encoding='utf-8')

# Choix des colonnes rélévantes
df_CPI_selected = df_CPI[['geo', 'TIME_PERIOD', 'OBS_VALUE']]

# Obtention des données de panel en format séries temporelles
df_CPI_pivot = df_CPI_selected.pivot_table(index='TIME_PERIOD', columns='geo', values='OBS_VALUE', aggfunc='first')

# Renommer les colonnes pour inclure le préfixe CPI_ (=Consumer price index)
df_CPI_pivot.columns = [f'CPI_{col}' for col in df_CPI_pivot.columns]

# On filtre les pays qui nous intéressent

# Vérification et filtrage
df_CPI=df_CPI_pivot
colonnes_a_garder = [f'CPI_{pays}' for pays in countries_selected]
df_CPI = df_CPI[[col for col in df_CPI.columns if col in colonnes_a_garder]]
df_CPI



In [ ]:
#Mise en place du CPI trimestriel
# Conversion de TIME_PERIOD en format datetime avec fréquence trimestrielle
df_CPI.index = pd.PeriodIndex(df_CPI.index, freq='Q')

# On calcule la moyenne par trimestre
df_CPI_trimestriel = df_CPI.groupby(df_CPI.index).mean()

# Changement du nom de l'index
df_CPI_trimestriel.index = df_CPI_trimestriel.index.astype(str).str.replace(r'(\d{4})Q(\d)', r'\1-Q\2', regex=True)
df_CPI=df_CPI_trimestriel
df_CPI


In [ ]:
# Concaténer les DataFrames
df_concat = pd.concat([df_GDP, df_CPI,df_LT_IR, df_ST_IR, df_working_hours], axis=1)

# Vérification des colonnes disponibles
print(df_concat.columns)

#  Réorganiser les colonnes pour que celles de chaque pays soient côte à côte
ordered_columns = []
for country in countries_selected:
    # Vérification que la colonne existe avant de l'ajouter
    if f'PIB_{country}' in df_concat.columns:
        ordered_columns.append(f'PIB_{country}')
    if f'CPI_{country}' in df_concat.columns:
        ordered_columns.append(f'CPI_{country}')
    if f'LT_IR_{country}' in df_concat.columns:
        ordered_columns.append(f'LT_IR_{country}')
    if f'ST_IR_{country}' in df_concat.columns:
        ordered_columns.append(f'ST_IR_{country}')
    if f'Working_Hours_{country}' in df_concat.columns:
        ordered_columns.append(f'Working_Hours_{country}')

# Réorganiser les colonnes
df_final = df_concat[ordered_columns]

# Afficher le DataFrame final
print(df_final)
